
**[PT]** Português

---

**[EN]** English


# Correção, inferência e normalização do campo "Faculdade"


---


# Correction, inference and normalization of the field "Faculdade"



## Setup

In [1]:
from timelink.api.database import TimelinkDatabase
from ucalumni.config import default_db_url

print(f"Creating TimelinkDatabase instance from {default_db_url}")
db = TimelinkDatabase(db_url=default_db_url)
# share with other notebooks
default_db_url


Creating TimelinkDatabase instance from sqlite:///../database/sqlite3/fauc.db?check_same_thread=False


'sqlite:///../database/sqlite3/fauc.db?check_same_thread=False'

## Notas de processamento

As notas de processamento são criadas pelo algoritmo de extração do valor da Faculdade.

Contém informação sobre inferências, correções automáticas e erros que necessitam de revisão.

Em termos narrativos, o algoritmo verifica primeiro se o campo "Faculdade" tem ou não um valor; depois procede à recolha de todas as referências a faculdades no resto do registo; se ao remover os pré-requisitos da recolha algo permanece, será o valor escolhido como faculdade; se nada permanece, então o valor original é usado; alguns outros casos especiais são testados. 


--- 

## Processing notes

The processing notes are created to register inferences, corrections and errors that need revision.

In narrative terms, the algorithm checks first if the field “Faculdade” has a value or not; it then proceeds to collect all references to “faculdades” in the rest of the record; if by removing the pre-requisites from the collection something remains, it will the chosen as the corrected or inferred “faculdade”; if nothing remains then the original value is used; some other edge cases are tested. 




In [3]:
from timelink.pandas import entities_with_attribute

notas_proc = entities_with_attribute(
                    the_type='nota-processamento',
                    entity_type='person',
                    show_elements=['name'],
                    more_attributes=['faculdade','faculdade-original','uc-entrada', 'uc-saida','url'],
                    db=db,
                    sql_echo=False)

notas_proc['id'] = notas_proc.index.values
notas_proc.dropna(how='all', axis=1, inplace=True)
notas_proc.info()

<class 'pandas.core.frame.DataFrame'>
Index: 25025 entries, 127766 to 357161
Data columns (total 19 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   name                      25025 non-null  object
 1   nota-processamento.type   25025 non-null  object
 2   nota-processamento        25025 non-null  object
 3   nota-processamento.date   25025 non-null  object
 4   nota-processamento.line   25025 non-null  int64 
 5   nota-processamento.level  25025 non-null  int64 
 6   nota-processamento.obs    25025 non-null  object
 7   faculdade                 24350 non-null  object
 8   faculdade.date            24350 non-null  object
 9   faculdade.obs             24350 non-null  object
 10  faculdade-original        12243 non-null  object
 11  faculdade-original.date   12243 non-null  object
 12  uc-entrada                25025 non-null  object
 13  uc-entrada.date           25025 non-null  object
 14  uc-saida             

In [4]:
df = notas_proc
df['id'] = df.index.values

col = 'nota-processamento' # sub total by this column
df_totals = df.groupby(col).agg({'id':'nunique',
                                                  'uc-entrada':'min',
                                                  'uc-saida':'max'})
df_totals.loc['Total'] = df_totals['id'].sum()

df_totals.sort_values('id',ascending= False).head(30)

,id,uc-entrada,uc-saida
nota-processamento,,,
Total,20263,20263,20263
Aviso: faculdade inferida,11356,0000-00-00,1913-07-11
Aviso: faculdade corrigida,6980,0000-00-00,1916-07-15
"Faculdade corrigida, combinação incomum",735,0000-00-00,1916-07-15
Erro: não foi possível determinar a faculdade,675,1537-00-00,1912-08-02
Erro: valor não corresponde a Faculdade nesta data,517,0000-00-00,1913-08-13



### Faculdade inferida

Casos em que o campo da faculdade não foi preenchido e o algoritmo sugere um valor.

---

### "Faculdade" Inferred


Cases where the value for faculty was missing and the algoritm suggested a value.



In [5]:
fac_inf = notas_proc[notas_proc['nota-processamento']=='Aviso: faculdade inferida'].copy()
fac_inf.dropna(how='all', axis=1, inplace=True)
fac_inf.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11759 entries, 127775 to 319862
Data columns (total 17 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   name                      11759 non-null  object
 1   nota-processamento.type   11759 non-null  object
 2   nota-processamento        11759 non-null  object
 3   nota-processamento.date   11759 non-null  object
 4   nota-processamento.line   11759 non-null  int64 
 5   nota-processamento.level  11759 non-null  int64 
 6   nota-processamento.obs    11759 non-null  object
 7   faculdade                 11759 non-null  object
 8   faculdade.date            11759 non-null  object
 9   faculdade.obs             11759 non-null  object
 10  uc-entrada                11759 non-null  object
 11  uc-entrada.date           11759 non-null  object
 12  uc-saida                  11759 non-null  object
 13  uc-saida.date             11759 non-null  object
 14  url                  

In [6]:
df = fac_inf
col = 'faculdade' # subotal by this column
df_totals = df.groupby(col).agg({'id':'nunique',
                                                  'uc-entrada':'min',
                                                  'uc-saida':'max'})
print("List of inferred 'faculdade'")
df_totals.sort_values('id',ascending= False).head(30)

List of inferred 'faculdade'


,id,uc-entrada,uc-saida
faculdade,,,
Cursos jurídicos (Cânones ou Leis),6993,0000-00-00,1829-03-28
Artes,3005,0000-00-00,1780-07-18
Cânones,764,0000-00-00,1827-06-20
Leis,386,0000-00-00,1828-07-01
Teologia,352,0000-00-00,1884-05-26
Medicina,185,0000-00-00,1883-10-15
Filosofia,28,1770-10-01,1913-07-11
Matemática,24,1605-10-07,1913-07-11
Direito,22,1836-10-14,1898-12-18


### Faculdade corrigida

Casos em que o campo da faculdade estava preenchido mas era inconsistente com o resto da informação.

A maior parte dos casos dizem respeito a duplo percursos em Leis e Cânones e mudanças, depois de 1772, 
de faculdades de Matemática e Filosofia, que eram pre-requisitos, para as faculdades de posterior
matrículas (Leis, Cânones, Medicina, Teologia).

---

### "Faculdade" corrected

Cases where the algoritm changed the value of the field, because it was inconsistent with
the rest of information on the record. Mostly double degres in Canon and Civil law, and
changes from pre-requisite faculty to final graduation faculty.


In [7]:
fac_corr = notas_proc[notas_proc['nota-processamento']=='Aviso: faculdade corrigida'].copy()
fac_corr.dropna(how='all', axis=1, inplace=True)

In [ ]:
import pandas as pd

df = fac_corr
col = 'nota-processamento.obs' # subotal by this column
df_totals = df.groupby(col).agg({'id':'nunique',
                                        'uc-entrada':'min',
                                        'uc-saida':'max'})
total = df_totals['id'].sum()
print("List of corrected 'faculdade'")
print("Total:",total)
df_totals['perc'] = df_totals['id']/total
pd.set_option('display.max_rows',1000)
df_totals.sort_values('id',ascending=False, inplace=True)
df_totals.head(10)

List of corrected 'faculdade'
Total: 6980


,id,uc-entrada,uc-saida,perc
nota-processamento.obs,,,,
"Cânones para Cânones,Leis.",1254,1536-10-00,1821-11-12,0.179656
"Leis para Cânones,Leis.",1246,1537-00-00,1829-06-13,0.178510
Matemática para Medicina.,1175,1772-11-28,1909-11-11,0.168338
Direito para Leis.,1023,1687-01-05,1827-06-16,0.146562
Filosofia para Medicina.,480,1540-10-25,1909-11-09,0.068768
"Teologia para Direito,Teologia.",232,1830-10-11,1915-10-22,0.033238
"Medicina para Cânones,Medicina.",208,1569-11-20,1778-10-20,0.029799
Direito para Cânones.,194,1566-10-07,1835-07-20,0.027794
"Cânones para Cânones,Teologia.",166,0000-00-00,1829-05-26,0.023782


In [14]:
fac_corr_nodup = fac_corr.drop_duplicates(subset=['id', 'faculdade-original'], keep='first')

In [17]:
show_only = 5
for  change,row in list(df_totals.iterrows())[:10]:
    print(f"{change}: {row.id}")
    for _ndx, row in fac_corr_nodup[fac_corr_nodup['nota-processamento.obs']==change][['id','name','url','uc-entrada']].head(show_only).iterrows():
        url = row.url.strip('"')
        print(f"     {row.id:6} {row['name']} {row['uc-entrada']} {url}")


Cânones para Cânones,Leis.: 1254
     127766 António Lopes Cabaço 1617-12-04 https://pesquisa.auc.uc.pt/details?id=127766
     127769 José Cabaço 1743-10-01 https://pesquisa.auc.uc.pt/details?id=127769
     127793 Miguel de Cabedo 1558-01-08 https://pesquisa.auc.uc.pt/details?id=127793
     128387 João Leite Cabral 1731-10-01 https://pesquisa.auc.uc.pt/details?id=128387
     128663 Francisco  de Caceres 1642-10-22 https://pesquisa.auc.uc.pt/details?id=128663
Leis para Cânones,Leis.: 1246
     127790 Jorge de Cabedo 1565-10-01 https://pesquisa.auc.uc.pt/details?id=127790
     128146 Domingos de Figueiredo Cabral 1682-10-01 https://pesquisa.auc.uc.pt/details?id=128146
     128242 Francisco Xavier da Silva Cabral 1776-10-29 https://pesquisa.auc.uc.pt/details?id=128242
     128247 Gaspar Cabral 1639-10-17 https://pesquisa.auc.uc.pt/details?id=128247
     128251 Gaspar Lopes Cabral 1642-10-25 https://pesquisa.auc.uc.pt/details?id=128251
Matemática para Medicina.: 1175
     127807 Manuel Joa

### "Valor não corresponde a Faculdade nesta data"

Esta lista é produzida pela comparação do valor do campo de faculdade, ou outra referência a faculdade no registo, e a lista das faculdades existentes à data do registo.

Para além dos erros de ortografia e gralhas (atenção a "Cãnones" com til), também contém:

* Uma grande quantidade (>300) de campos de Faculdade com "?". Verificar se é porque falta informação na ficha, ou se existe outra explicação
* valores confundidos com Faculdade por particularidades da pontuação do registo. Nesse caso a correção a fazer pode ser na pontuação. 
   Ver por exemplo: 152975 a falta de ":" a seguir a matrícula provoca um erro no processamento da Faculdade
* valor que correspondem a faculdades que existiram, mas não naquelas datas (por exemplo Filosofia antes de 1772 ou Cânones depois de 1834).
* Topónimos, que serão erros de digitação que se poderão corrigir vendo a ficha original.

---

### Value does not correspond to a "Faculdade" at the time

This list is produced by comparing the value of the faculty field, or other reference to faculties in the record, and the list of faculties existing at the date of the record.

Apart from spelling errors and typos (watch out for tilted "Cãnones"), it also contains:

* A large amount (>300) of Faculty fields with "?". Check if this is because the information is missing in the original card, or if there is another explanation.
* Values confused with Faculty due to particularities of the register punctuation. In this case, the correction to be made may be in the punctuation. 
   See for example: 152975 the lack of ":" after the enrolment causes an error in the processing of the Faculty
* value corresponding to faculties that existed but not on those dates (e.g. Philosophy before 1772 or "Cânones" after 1834).
* Toponyms, which are typing errors that can be corrected by looking at the original form.

In [18]:
fac_error = notas_proc[notas_proc['nota-processamento']=='Erro: valor não corresponde a Faculdade nesta data'].copy()
fac_error.info()

<class 'pandas.core.frame.DataFrame'>
Index: 519 entries, 130413 to 317873
Data columns (total 17 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   name                     519 non-null    object
 1   sex                      519 non-null    object
 2   nota-processamento       519 non-null    object
 3   nota-processamento.date  519 non-null    object
 4   nota-processamento.obs   519 non-null    object
 5   faculdade                519 non-null    object
 6   faculdade.date           519 non-null    object
 7   faculdade.obs            519 non-null    object
 8   faculdade-original       206 non-null    object
 9   faculdade-original.date  206 non-null    object
 10  uc-entrada               519 non-null    object
 11  uc-entrada.date          519 non-null    object
 12  uc-saida                 519 non-null    object
 13  uc-saida.date            519 non-null    object
 14  url                      519 non-null  

In [19]:
# currently the algorithm removed question marks, but they were there
# we have to put them back until this is fixed
fac_org_none = fac_error['faculdade-original'].isnull()
fac_error.loc[fac_org_none,'faculdade-original'] = '?'


In [20]:
df = fac_error
col = 'faculdade-original' # subotal by this column
df_totals = df.groupby(col).agg({'id':'nunique',
                                                  'uc-entrada':'min',
                                                  'uc-saida':'max'})

df_totals.sort_values('id',ascending= False).head(30)

,id,uc-entrada,uc-saida
faculdade-original,,,
?,313,0000-00-00,1771-01-01
Leis,23,1827-10-13,1909-08-03
Cânones,22,1825-10-24,1913-08-13
Cãnones,21,1617-11-29,1769-07-10
Medecina,18,1654-10-15,1858-10-12
.,13,1604-10-17,1732-11-10
Intituta,8,1581-03-12,1770-10-01
Instituto,7,1694-12-15,1818-10-24
Medicna,5,1590-10-08,1735-05-08


In [24]:
import pandas as pd
import csv

pd.set_option('display.max_rows',600)
exportar = fac_error[['faculdade','name','faculdade.date']].sort_values(['faculdade','name','faculdade.date'])
exportar.to_csv('../inferences/validation/faculdade-unkown.csv',sep=',')
exportar.head(10)

,faculdade,name,faculdade.date
id,,,
192086,('),António Rodrigues,1626-10-29
146645,(.),António Antunes,1656-10-20
140542,(.),António de Abreu,1661-10-26
165702,(.),Manuel Ribeiro,1604-10-17
165556,(.),Manuel Ribeiro,1612-03-23
165558,(.),Manuel Ribeiro,1617-04-09
165781,(.),Manuel Ribeiro,1657-10-15
165804,(.),Manuel Ribeiro,1659-12-12
165730,(.),Manuel Ribeiro,1683-03-24


### "Faculdades" com menos de 100 ocorrências. 

---


### "Faculdades" with less than 100 stduents

In [26]:

from timelinknb.pandas import attribute_values


facs = attribute_values('faculdade',dates_between=('1500-00-00','1990-00-00'))
facs[facs['count']<100]

,count,date_in,date_max
value,,,
(Leis),33,1819-10-16,1906-10-02
(Cânones),32,1822-10-15,1909-10-02
(Cãnones),24,1572-10-01,1769-05-16
(Medecina),19,1654-10-15,1858-10-12
(.),16,1604-10-17,1738-12-01
código,12,1540-00-00,1540-12-16
(Intituta),8,1581-03-12,1770-10-01
(Instituto),7,1694-12-15,1818-10-24
(Medicna),7,1573-10-03,1745-10-01


In [33]:
facs_errors = facs[facs['count']<100]
show_only = 10
for f,row in facs_errors.sort_values('count',ascending=False).iterrows():
        print()
        print(f)
        students = attribute_to_df('faculdade',the_value=f,person_info=True)
        for id, aluno in students.head(show_only).iterrows():
                id_link = f'<a href="https://pesquisa.auc.uc.pt/details?id={id}">{id}</a>'
                # display(HTML(f'<code>{row.the_value:24} [{id_link:5}] {aluno.the_date} {aluno.name:44}</code>'))
                print(f"{aluno['faculdade']:24} [{id:5}] {aluno['faculdade.date']} {aluno['name']:44} https://pesquisa.auc.uc.pt/details?id={id} {aluno['faculdade.obs']}")



(Leis)
(Leis)                   [188758] 1819-10-16 Manuel Joaquim de Sousa Brito                https://pesquisa.auc.uc.pt/details?id=188758 Faculdade corrigida
(Leis)                   [215793] 1822-10-08 António Pedro Mousinho Leote                 https://pesquisa.auc.uc.pt/details?id=215793 Faculdade corrigida
(Leis)                   [141618] 1823-10-07 Miguel Luís Henriques de Aguiar              https://pesquisa.auc.uc.pt/details?id=141618 Faculdade corrigida
(Leis)                   [210323] 1824-10-30 Francisco de Meireles Leite                  https://pesquisa.auc.uc.pt/details?id=210323 Faculdade corrigida
(Leis)                   [128134] 1825-10-14 Constantino António do Vale Pereira Cabral   https://pesquisa.auc.uc.pt/details?id=128134 Faculdade corrigida
(Leis)                   [167725] 1825-10-31 Justino António de Freitas                   https://pesquisa.auc.uc.pt/details?id=167725 Faculdade corrigida
(Leis)                   [145767] 1826-10-14 Henriques Teles d